In [1]:
import config
from Utils.hdf5DatasetWriter import HDF5DatasetWriter
import numpy as np

/home/shared/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
f = open(config.INPUT_PATH)
# skip the header of csv file
f.__next__()

(trainImages, trainLabels) = ([], [])
(valImages, valLabels) = ([], [])
(testImages, testLabels) = ([], [])

for row in f:
    (label, image, usage) = row.strip().split(',')
    
    label = int(label)
    if label == 1:
        label = 0
    if label > 0:
        label -= 1
    
    image = np.array(image.split(' '), dtype='uint8')
    image = image.reshape((48, 48))
    
    if usage == 'Training':
        trainImages.append(image)
        trainLabels.append(label)
    elif usage == 'PrivateTest':
        valImages.append(image)
        valLabels.append(label)
    else:
        testImages.append(image)
        testLabels.append(label)

In [6]:
dataset = [[config.TRAIN_HDF5, trainImages, trainLabels],
           [config.VAL_HDF5, valImages, valLabels],
           [config.TEST_HDF5, testImages, testLabels]]

for (outputPath, images, labels) in dataset:
    writer = HDF5DatasetWriter((len(images), 48, 48), outputPath)
    for (image, label) in zip(images, labels):
        writer.add([image], [label])
    writer.close()

f.close()